In [2]:
# !pip install --quiet  datasets #to access squad dataset
# !pip install --quiet pyarrow   #to deal with parquet files for saving dataset if required
# !pip install --quiet  tqdm     #for progress bars
# !pip install --quiet transformers # for t5 model
# !pip install --quiet tokenizers  #tokenizers from HuggingFace
# !pip install --quiet sentencepiece #subword tokenizer used by T5
# !pip install --quiet pytorch-lightning # pytorch wrapper 
# !pip install --quiet torchtext # text utilities

# Load dataset

In [3]:
#imports
import pandas as pd
import torch
from tqdm import tqdm
from datasets import DatasetDict, Dataset, load_from_disk
# from torch.utils.data import Dataset, DataLoader
from pprint import pprint
import copy, os, json, shutil
import numpy as np
from collections import defaultdict
import ipdb, re

pd.options.display.max_rows , pd.options.display.max_columns  = 100,100  

# Fraction for sample dataframe
proportion = 0.5

In [4]:
arxiv_txt_summarised_dec092023 = os.listdir("/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/arxiv_txt_summarised_dec092023")
arxiv_txt_dec092023 = os.listdir("/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/arxiv_txt_dec092023")

missed_lb_papers = []
for paper in arxiv_txt_summarised_dec092023:
    paper_id = paper.rsplit("_",1)[0]
    if f"{paper_id}.txt" in arxiv_txt_dec092023:
        # paper_id_full =  paper.rsplit(".",1)[0]
        pass
    else:
        missed_lb_papers.append(paper_id)


print(f"Missed Leaderboard papers: {len(missed_lb_papers)}")

arxiv_txt_summarised_dec092023 = os.listdir("/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/arxiv_no_leaderboard_txt_25_000_summarised_dec092023")
arxiv_txt_dec092023 = os.listdir("/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/arxiv_no_leaderboard_txt_25_000_dec092023")

missed_no_lb_papers = []
for paper in arxiv_txt_summarised_dec092023:
    paper_id = paper.rsplit("_",1)[0]
    if f"{paper_id}.txt" in arxiv_txt_dec092023:
        # paper_id_full =  paper.rsplit(".",1)[0]
        pass
    else:
        missed_no_lb_papers.append(paper_id)


print(f"Missed No Leaderboard papers: {len(missed_no_lb_papers)}")

Missed Leaderboard papers: 289
Missed No Leaderboard papers: 911


In [5]:
path_leaderboard_tex = "/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/arxiv_tex_dec092023"
target_lb_folder = "/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/retry_tex_lb"

for latex_paper in tqdm(missed_lb_papers):
    if not os.path.exists(f"{target_lb_folder}/{latex_paper}.tex"):
        shutil.copyfile(f"{path_leaderboard_tex}/{latex_paper}.tex", f"{target_lb_folder}/{latex_paper}.tex")
    
path_no_leaderboard_tex = "/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/arxiv_no_leaderboard_tex_25_000"
target_no_lb_folder = "/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/retry_tex_no_lb"

for latex_paper in tqdm(missed_no_lb_papers):
    if not os.path.exists(f"{target_no_lb_folder}/{latex_paper}.tex"):
        shutil.copyfile(f"{path_no_leaderboard_tex}/{latex_paper}.tex", f"{target_no_lb_folder}/{latex_paper}.tex")

100%|██████████| 911/911 [00:00<00:00, 8841.51it/s]


In [6]:
# !pwd

In [7]:
# %cd ../data_proccess/
!pwd
# !bash /nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/tex_to_txt.sh /nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/retry_tex_lb /nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/arxiv_txt_dec092023
# !bash tex_to_txt.sh retry_tex_lb arxiv_txt_dec092023
# !bash tex_to_txt.sh retry_tex_no_lb arxiv_no_leaderboard_txt_25_000_dec092023

/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/notebooks


In [8]:
# Long txt 
base_txt_path = "/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess"
# "/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/arxiv_no_leaderboard_txt_25_000_dec092023"

# AUGMENTED
train_f1_data_path = "../data/df_train_tdms_augmented_summarized_with_id_f1_all_templates.parquet"
dev_f1_data_path = "../data/df_dev_tdms_augmented_summarized_with_id_f1_all_templates.parquet"
zeroshot_f1_data_path = "../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_all_templates.parquet"

train_f2_data_path = "../data/df_train_tdms_augmented_summarized_with_id_f2_all_templates.parquet"
dev_f2_data_path = "../data/df_dev_tdms_augmented_summarized_with_id_f2_all_templates.parquet"


df_train_tdms_f1 = pd.read_parquet(train_f1_data_path)
df_dev_tdms_f1 = pd.read_parquet(dev_f1_data_path)
df_zeroshot_tdms_f1 = pd.read_parquet(zeroshot_f1_data_path)

df_train_tdms_f2 = pd.read_parquet(train_f2_data_path)
df_dev_tdms_f2 = pd.read_parquet(dev_f2_data_path)

df_train_tdms_f1.tail()

,id,prompt,answer
185815,1111.2942.pdf,Title:\tDown the Rabbit Hole: Robust Proximity...,unanswerable
185816,1111.2942.pdf,Title:\tDown the Rabbit Hole: Robust Proximity...,unanswerable
185817,1111.2942.pdf,Read this article and answer this question Tit...,unanswerable
185818,1111.2942.pdf,Title:\tDown the Rabbit Hole: Robust Proximity...,unanswerable
185819,1111.2942.pdf,Context: Title:\tDown the Rabbit Hole: Robust ...,unanswerable


In [9]:
df_zeroshot_tdms_f1.tail()

,id,prompt,answer
17995,1006.1165.pdf,Title:\t\n\nAbstract:\t\n\nA Data Structure fo...,unanswerable
17996,1006.1165.pdf,Title:\t\n\nAbstract:\t\n\nA Data Structure fo...,unanswerable
17997,1006.1165.pdf,Read this article and answer this question Tit...,unanswerable
17998,1006.1165.pdf,Title:\t\n\nAbstract:\t\n\nA Data Structure fo...,unanswerable
17999,1006.1165.pdf,Context: Title:\t\n\nAbstract:\t\n\nA Data Str...,unanswerable


In [10]:
'Please answer a question about Answer this question:  this article. If the question is Answer this question: unanswerable, say "unanswerable".'.replace('Answer this question: ', '')

'Please answer a question about  this article. If the question is unanswerable, say "unanswerable".'

In [11]:
def find_template(txts):
    
    # Filter rows where the 'text' column contains the given substring
    # substring_to_match_squad_1 = re.escape('Please answer a question about this article. If the question is unanswerable, say "unanswerable".')
    
    substring_to_match_squad_1 = 'Please answer a question about this article. If the question is unanswerable, say "unanswerable".'
    substring_to_match_squad_2 = 'Read this and answer the question. If the question is unanswerable, say \"unanswerable\".'
    substring_to_match_squad_3 = '(If the question is unanswerable, say \"unanswerable\"'
    substring_to_match_squad_4 = 'Try to answer this question if possible (otherwise reply \"unanswerable\")'
    substring_to_match_squad_5 = 'If it is possible to answer this question, answer it for me (else, reply \"unanswerable\"):'
    substring_to_match_squad_6 = 'Answer this question, if possible (if impossible, reply \"unanswerable\"):'
    substring_to_match_squad_7 = 'What is the answer? (If it cannot be answered, return \"unanswerable\")'
    substring_to_match_squad_8 = 'Now answer this question, if there is an answer (If it cannot be answered, return \"unanswerable\"):'

    substring_to_match_drop_1 = 'Answer based on context:'
    substring_to_match_drop_2 = 'Answer this question based on the article:'
    # substring_to_match_drop_3 = ''
    substring_to_match_drop_4 = 'Answer this question: '
    substring_to_match_drop_5 = 'Read this article and answer this question'
    substring_to_match_drop_6 = 'Based on the above article, answer a question.'
    substring_to_match_drop_7 = 'Context: '

    template = "squad_1" if substring_to_match_squad_1 in txts else \
            "squad_2" if substring_to_match_squad_2 in txts else \
            "squad_3" if substring_to_match_squad_3 in txts else \
            "squad_4" if substring_to_match_squad_4 in txts else \
            "squad_5" if substring_to_match_squad_5 in txts else \
            "squad_6" if substring_to_match_squad_6 in txts else \
            "squad_7" if substring_to_match_squad_7 in txts else \
            "squad_8" if substring_to_match_squad_8 in txts else \
            "drop_1" if substring_to_match_drop_1 in txts else \
            "drop_2" if substring_to_match_drop_2 in txts else \
            "drop_4" if substring_to_match_drop_4 in txts else \
            "drop_5" if substring_to_match_drop_5 in txts else \
            "drop_6" if substring_to_match_drop_6 in txts else \
            "drop_7" if substring_to_match_drop_7 in txts else "drop_3"
            
    return template

def add_template_df(df):
        # Filter rows where the 'text' column contains the given substring
        # substring_to_match_squad_1 = re.escape('Please answer a question about this article. If the question is unanswerable, say "unanswerable".')
        substring_to_match_squad_1 = 'Please answer a question about this article. If the question is unanswerable, say "unanswerable".'
        substring_to_match_squad_2 = 'Read this and answer the question. If the question is unanswerable, say \"unanswerable\".'
        substring_to_match_squad_3 = '(If the question is unanswerable, say \"unanswerable\"'
        substring_to_match_squad_4 = 'Try to answer this question if possible (otherwise reply \"unanswerable\")'
        substring_to_match_squad_5 = 'If it is possible to answer this question, answer it for me (else, reply \"unanswerable\"):'
        substring_to_match_squad_6 = 'Answer this question, if possible (if impossible, reply \"unanswerable\"):'
        substring_to_match_squad_7 = 'What is the answer? (If it cannot be answered, return \"unanswerable\")'
        substring_to_match_squad_8 = 'Now answer this question, if there is an answer (If it cannot be answered, return \"unanswerable\"):'

        substring_to_match_drop_1 = 'Answer based on context:'
        substring_to_match_drop_2 = 'Answer this question based on the article:'
        # substring_to_match_drop_3 = ''
        substring_to_match_drop_4 = 'Answer this question: '
        substring_to_match_drop_5 = 'Read this article and answer this question'
        substring_to_match_drop_6 = 'Based on the above article, answer a question.'
        substring_to_match_drop_7 = 'Context: '

        # TDMS
        df['template'] = df['prompt'].apply(
        lambda x : "squad_1" if substring_to_match_squad_1 in x else \
                "squad_2" if substring_to_match_squad_2 in x else \
                "squad_3" if substring_to_match_squad_3 in x else \
                "squad_4" if substring_to_match_squad_4 in x else \
                "squad_5" if substring_to_match_squad_5 in x else \
                "squad_6" if substring_to_match_squad_6 in x else \
                "squad_7" if substring_to_match_squad_7 in x else \
                "squad_8" if substring_to_match_squad_8 in x else \
                "drop_1" if substring_to_match_drop_1 in x else \
                "drop_2" if substring_to_match_drop_2 in x else \
                "drop_4" if substring_to_match_drop_4 in x else \
                "drop_5" if substring_to_match_drop_5 in x else \
                "drop_6" if substring_to_match_drop_6 in x else \
                "drop_7" if substring_to_match_drop_7 in x else "drop_3"
                )
        return df

def df_stats(df):
  df = add_template_df(df)
  display(df.describe())
  df["prompt lenght"] = df.prompt.apply(lambda x: len(x.split()))
  display(df.describe())

In [12]:
# df_stats(df_train_tdms_f1)

In [13]:
# df_train_tdms_f1.iloc[200]['id'].rsplit(".", 1)

In [14]:
# df_train_tdms_f1 = pd.read_parquet(train_f1_data_path)
# df_dev_tdms_f1 = pd.read_parquet(dev_f1_data_path)
# df_zeroshot_tdms_f1 = pd.read_parquet(zeroshot_f1_data_path)

# df_train_tdms_f2 = pd.read_parquet(train_f2_data_path)
# df_dev_tdms_f2 = pd.read_parquet(dev_f2_data_path)

# DOCTEAT

In [15]:
# DOCTEAT
docteat_lb_path = "/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/arxiv_xml_lb_dec092023_DOCTEAT"
docteat_no_lb_path = "/nfs/home/kabenamualus/Research/LLLM-LeaderboardLLM/data_proccess/arxiv_xml_no_lb_dec092023_DOCTEAT"
dict_docteat = {}

df_docteat_lb = pd.read_csv(f"{docteat_lb_path}/DocTAET_full.tsv", delimiter='\t', names=["id", "context"])
df_docteat_no_lb = pd.read_csv(f"{docteat_no_lb_path}/DocTAET_full.tsv", delimiter='\t', names=["id", "context"])

In [16]:
df_docteat_lb.describe()

,id,context
count,10327,10327
unique,10327,10102
top,2106.10846v1.pdf,LaTeX Author Guidelines for ICCV Proceedings T...
freq,1,41


In [17]:
records = df_docteat_lb.to_dict("records")

for i, row in tqdm(enumerate(records), total = len(records)):
    dict_docteat[row['id']] = row['context']

100%|██████████| 10327/10327 [00:00<00:00, 2165295.81it/s]


In [18]:
df_docteat_no_lb.describe()

,id,context
count,8675,8675
unique,8675,8667
top,1008.5248.pdf,Bare Advanced Demo of IEEEtran.clsfor Computer...
freq,1,5


In [19]:
records = df_docteat_no_lb.to_dict("records")

for i, row in tqdm(enumerate(records), total = len(records)):
    dict_docteat[row['id']] = row['context']

100%|██████████| 8675/8675 [00:00<00:00, 2055218.44it/s]


In [20]:
def create_pandas_new_context_docteat(df, no_template=False):

  ''' Create a Pandas Dataframe from pandas.
  Params:
        answer_threshold: Only consider those Question Answer pairs where the Answer is short.
  '''
  count_index = 0
  result_df  = pd.DataFrame(columns = ['id', 'prompt', 'answer'])   
  
  q_types = [
    {"q": "What are the values for the following properties to construct a Leaderboard for the model introduced in this article: task, dataset, metric, and score?", "a_key": "answer"},
    ]
  
  records = df.to_dict("records")
  missed_parsing = []
  
  for i, row in tqdm(enumerate(records), total = len(records)):        
      for q_type in q_types:
        
        template = find_template(row["prompt"])
        file_id = row["id"].rsplit(".", 1)[0]
        try:
          
          new_context = dict_docteat[f'{file_id}.pdf']
    
        except :
          missed_parsing.append(file_id)
          continue
        
        if no_template:
          result_df.loc[count_index] = [str(row["id"])] + [f'{new_context}\n\n". {q_type["q"]}'] \
            + [str(row[q_type["a_key"]])]
          count_index += 1
          continue
                
        ## Squad_v2 
        if template == "squad_1":
          # prompt = row["prompt"].replace(
          #   "\nPlease answer a question about this article. If the question is unanswerable, say \"unanswerable\".",
          #   ""
          # ).replace(
          #   "What are the values for the following properties to construct a Leaderboard for the model introduced in this article: task, dataset, metric, and score?",
          #   ""
          # )
          
          result_df.loc[count_index] = [str(row["id"])] + [f'{new_context}\n\nPlease answer a question about this article. If the question is unanswerable, say \"unanswerable\". {q_type["q"]}'] \
            + [str(row[q_type["a_key"]])]
          count_index += 1
        
        if template == "squad_2":
          result_df.loc[count_index] = [str(row["id"])] + [f'Read this and answer the question. If the question is unanswerable, say \"unanswerable\".\n\n{new_context}\n\n{q_type["q"]}'
  ] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if template == "squad_3":
          result_df.loc[count_index] = [str(row["id"])] + [f'{new_context}\n{q_type["q"]} (If the question is unanswerable, say \"unanswerable\"'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if template == "squad_4":
          result_df.loc[count_index] = [str(row["id"])] + [f'{new_context}\nTry to answer this question if possible (otherwise reply \"unanswerable\"): {q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if template == "squad_5":
          result_df.loc[count_index] = [str(row["id"])] + [f'{new_context}\nIf it is possible to answer this question, answer it for me (else, reply \"unanswerable\"): {q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if template == "squad_6":
          result_df.loc[count_index] = [str(row["id"])] + [f'{new_context}\n\nAnswer this question, if possible (if impossible, reply \"unanswerable\"): {q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if template == "squad_7":
          result_df.loc[count_index] = [str(row["id"])] + [f'Read this: {new_context}\n\n{q_type["q"]}\nWhat is the answer? (If it cannot be answered, return \"unanswerable\")'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if template == "squad_8":
          result_df.loc[count_index] = [str(row["id"])] + [f'Read this: {new_context}\nNow answer this question, if there is an answer (If it cannot be answered, return \"unanswerable\"): {q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        
        # Drop
        if template == "drop_1":
          result_df.loc[count_index] = [str(row["id"])] + [f'Answer based on context:\n\n{new_context}\n\n{q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if template == "drop_2":
          result_df.loc[count_index] = [str(row["id"])] + [f'{new_context}\n\nAnswer this question based on the article: {q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if template == "drop_3":
          result_df.loc[count_index] = [str(row["id"])] + [f'{new_context}\n\n{q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if template == "drop_4":
          result_df.loc[count_index] = [str(row["id"])] + [f'{new_context}\nAnswer this question: {q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if template == "drop_5":
          result_df.loc[count_index] = [str(row["id"])] + [f'Read this article and answer this question {new_context}\n{q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if template == "drop_6":
          result_df.loc[count_index] = [str(row["id"])] + [f'{new_context}\n\nBased on the above article, answer a question. {q_type["q"]}'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
        
        if template == "drop_7":
          result_df.loc[count_index] = [str(row["id"])] + [f'Context: {new_context}\n\nQuestion: {q_type["q"]}\n\nAnswer:'] \
            + [str(row[q_type["a_key"]])] 
          count_index += 1
  
  print(f"Missed file parsing count: {len(missed_parsing)}")
  return result_df, missed_parsing

In [21]:
df_train_tdms_f1.head()

,id,prompt,answer
0,1707.03497v2.pdf,Title:\tValue Prediction Network\n\nAbstract:\...,"[{'LEADERBOARD': {'Task': 'Atari Games', 'Data..."
1,1707.03497v2.pdf,Read this and answer the question. If the ques...,"[{'LEADERBOARD': {'Task': 'Atari Games', 'Data..."
2,1707.03497v2.pdf,Title:\tValue Prediction Network\n\nAbstract:\...,"[{'LEADERBOARD': {'Task': 'Atari Games', 'Data..."
3,1707.03497v2.pdf,Title:\tValue Prediction Network\n\nAbstract:\...,"[{'LEADERBOARD': {'Task': 'Atari Games', 'Data..."
4,1707.03497v2.pdf,Title:\tValue Prediction Network\n\nAbstract:\...,"[{'LEADERBOARD': {'Task': 'Atari Games', 'Data..."


In [22]:
df_train_tdms_f1_docteat_no_template, missed_f1_train = create_pandas_new_context_docteat(df_train_tdms_f1, True)
df_dev_tdms_f1_docteat_no_template, missed_f1_dev = create_pandas_new_context_docteat(df_dev_tdms_f1, True)
df_zeroshot_tdms_f1_docteat_no_template, missed_f1_dev = create_pandas_new_context_docteat(df_zeroshot_tdms_f1, True)

df_train_tdms_f2_docteat_no_template, missed_f2_train = create_pandas_new_context_docteat(df_train_tdms_f2, True)
df_dev_tdms_f2_docteat_no_template, missed_f2_dev = create_pandas_new_context_docteat(df_dev_tdms_f2, True)

100%|██████████| 185820/185820 [13:21<00:00, 231.86it/s]


Missed file parsing count: 34950


100%|██████████| 24630/24630 [00:29<00:00, 830.79it/s] 


Missed file parsing count: 4425


100%|██████████| 18000/18000 [00:19<00:00, 927.53it/s] 


Missed file parsing count: 4110


100%|██████████| 185730/185730 [13:46<00:00, 224.77it/s]


Missed file parsing count: 34740


100%|██████████| 24720/24720 [00:29<00:00, 838.33it/s] 

Missed file parsing count: 4635


In [23]:
df_dev_tdms_f1_docteat_no_template.tail()

,id,prompt,answer
20200,1602.04536.pdf,Data Load Balancing in Heterogeneous Dynamic N...,unanswerable
20201,1602.04536.pdf,Data Load Balancing in Heterogeneous Dynamic N...,unanswerable
20202,1602.04536.pdf,Data Load Balancing in Heterogeneous Dynamic N...,unanswerable
20203,1602.04536.pdf,Data Load Balancing in Heterogeneous Dynamic N...,unanswerable
20204,1602.04536.pdf,Data Load Balancing in Heterogeneous Dynamic N...,unanswerable


In [24]:
df_train_tdms_f1_docteat, missed_f1_train = create_pandas_new_context_docteat(df_train_tdms_f1, False)
df_dev_tdms_f1_docteat, missed_f1_dev = create_pandas_new_context_docteat(df_dev_tdms_f1, False)
df_zeroshot_tdms_f1_docteat, missed_f1_dev = create_pandas_new_context_docteat(df_zeroshot_tdms_f1, False)

df_train_tdms_f2_docteat, missed_f2_train = create_pandas_new_context_docteat(df_train_tdms_f2, False)
df_dev_tdms_f2_docteat, missed_f2_dev = create_pandas_new_context_docteat(df_dev_tdms_f2, False)

100%|██████████| 185820/185820 [13:19<00:00, 232.35it/s]


Missed file parsing count: 34950


100%|██████████| 24630/24630 [00:29<00:00, 828.58it/s] 


Missed file parsing count: 4425


100%|██████████| 18000/18000 [00:19<00:00, 920.95it/s] 


Missed file parsing count: 4110


100%|██████████| 185730/185730 [13:30<00:00, 229.26it/s]


Missed file parsing count: 34740


100%|██████████| 24720/24720 [00:29<00:00, 840.31it/s] 

Missed file parsing count: 4635


In [25]:
df_dev_tdms_f1_docteat.tail()

,id,prompt,answer
20200,1602.04536.pdf,Data Load Balancing in Heterogeneous Dynamic N...,unanswerable
20201,1602.04536.pdf,Data Load Balancing in Heterogeneous Dynamic N...,unanswerable
20202,1602.04536.pdf,Read this article and answer this question Dat...,unanswerable
20203,1602.04536.pdf,Data Load Balancing in Heterogeneous Dynamic N...,unanswerable
20204,1602.04536.pdf,Context: Data Load Balancing in Heterogeneous ...,unanswerable


In [26]:
print("Train F1 Docteat")
df_stats(df_train_tdms_f1_docteat)
print("Train F1 Docteat No Template")
df_stats(df_train_tdms_f1_docteat_no_template)

print("Zeroshot F1 Docteat")
df_stats(df_zeroshot_tdms_f1_docteat)
print("Zeroshot F1 Docteat No Template")
df_stats(df_zeroshot_tdms_f1_docteat_no_template)

print("Dev F1 Docteat")
df_stats(df_dev_tdms_f1_docteat)
print("Dev F1 Docteat No Templates")
df_stats(df_dev_tdms_f1_docteat_no_template)

print("Train F2 Docteat")
df_stats(df_train_tdms_f2_docteat)
print("Train F2 Doctea No Templatest")
df_stats(df_train_tdms_f2_docteat_no_template)

print("Dev F2 Docteat")
df_stats(df_dev_tdms_f2_docteat)
print("Dev F2 Docteat No Templatest")
df_stats(df_dev_tdms_f2_docteat_no_template)

Train F1 Docteat


,id,prompt,answer,template
count,150870,150870,150870,150870
unique,10058,150053,7016,15
top,1707.03497v2.pdf,Author Guidelines for ECCV Submission The abst...,unanswerable,drop_7
freq,15,7,45495,10065


,prompt lenght
count,150870.000000
mean,493.440770
std,398.751372
min,26.000000
25%,224.000000
50%,400.000000
75%,648.000000
max,7361.000000


Train F1 Docteat No Template


,id,prompt,answer,template
count,150870,150870,150870,150870
unique,10058,10004,7016,1
top,1707.03497v2.pdf,Author Guidelines for ECCV Submission The abst...,unanswerable,drop_3
freq,15,105,45495,150870


...

,id,prompt,answer,template
count,20085,20085,20085,20085
unique,1339,20068,873,15
top,2006.10721v2.pdf,Read this: TFNet: Multi-Semantic Feature Inter...,unanswerable,drop_7
freq,15,2,7005,1341


,prompt lenght
count,20085.000000
mean,480.242221
std,355.462022
min,28.000000
25%,232.000000
50%,401.000000
75%,628.000000
max,5153.000000


Dev F2 Docteat No Templatest


,id,prompt,answer,template
count,20085,20085,20085,20085
unique,1339,1338,873,1
top,2006.10721v2.pdf,TFNet: Multi-Semantic Feature Interaction for ...,unanswerable,drop_3
freq,15,30,7005,20085


,prompt lenght
count,20085.000000
mean,472.508588
std,355.427904
min,29.000000
25%,224.000000
50%,392.000000
75%,619.000000
max,5136.000000


In [27]:
df_train_tdms_f1_docteat.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_all_templates_docteat.parquet')
df_dev_tdms_f1_docteat.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_all_templates_docteat.parquet')
df_zeroshot_tdms_f1_docteat.to_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_all_templates_docteat.parquet')

df_train_tdms_f2_docteat.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_all_templates_docteat.parquet')
df_dev_tdms_f2_docteat.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_all_templates_docteat.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_all_templates_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_all_templates_docteat.parquet'),
        "zeroshot": Dataset.from_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_all_templates_docteat.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_all_templates_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_all_templates_docteat.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_AUGMENTED_SUMMARIZED_ZEROSHOT_TDMS_ALL_TEMPLATE_DOCTEAT")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-daa76544d89a97f7/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

Saving the dataset (0/2 shards):   0%|          | 0/150870 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/20205 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13890 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/150990 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/20085 [00:00<?, ? examples/s]

In [28]:
df_train_tdms_f1_docteat_no_template.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_no_templates_docteat.parquet')
df_dev_tdms_f1_docteat_no_template.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_no_templates_docteat.parquet')
df_zeroshot_tdms_f1_docteat_no_template.to_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_no_templates_docteat.parquet')

df_train_tdms_f2_docteat_no_template.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_no_templates_docteat.parquet')
df_dev_tdms_f2_docteat_no_template.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_no_templates_docteat.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_no_templates_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_no_templates_docteat.parquet'),
        "zeroshot": Dataset.from_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_no_templates_docteat.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_no_templates_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_no_templates_docteat.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_AUGMENTED_SUMMARIZED_ZEROSHOT_TDMS_NO_TEMPLATE_DOCTEAT")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /nfs/home/kabenamualus/.cache/huggingface/datasets/parquet/default-7bd603c1b3e49457/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

Saving the dataset (0/2 shards):   0%|          | 0/150870 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/20205 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13890 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/150990 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/20085 [00:00<?, ? examples/s]

# Specific Template

In [33]:
df_train_tdms_f1_docteat[df_train_tdms_f1_docteat.template == "squad_1"].describe()

,prompt lenght
count,10058.000000
mean,498.707298
std,398.739365
min,40.000000
25%,229.000000
50%,405.000000
75%,653.000000
max,7357.000000


In [30]:
df_train_tdms_f1_docteat_no_template.tail()

,id,prompt,answer,template,prompt lenght
150865,1111.2942.pdf,Down the Rabbit Hole: Robust Proximity Search ...,unanswerable,drop_3,232
150866,1111.2942.pdf,Down the Rabbit Hole: Robust Proximity Search ...,unanswerable,drop_3,232
150867,1111.2942.pdf,Down the Rabbit Hole: Robust Proximity Search ...,unanswerable,drop_3,232
150868,1111.2942.pdf,Down the Rabbit Hole: Robust Proximity Search ...,unanswerable,drop_3,232
150869,1111.2942.pdf,Down the Rabbit Hole: Robust Proximity Search ...,unanswerable,drop_3,232


In [34]:
df_train_f1_squad_1 = df_train_tdms_f1_docteat[df_train_tdms_f1_docteat.template == "squad_1"]
                 
df_dev_f1_squad_1 = df_dev_tdms_f1_docteat[df_dev_tdms_f1_docteat.template == "squad_1"]

df_zeroshot_f1_squad_1 = df_zeroshot_tdms_f1_docteat[df_zeroshot_tdms_f1_docteat.template == "squad_1"]

df_train_f2_squad_1 = df_train_tdms_f2_docteat[df_train_tdms_f2_docteat.template == "squad_1"]
df_dev_f2_squad_1 = df_dev_tdms_f2_docteat[df_dev_tdms_f2_docteat.template == "squad_1"]

print("df_train_f1_squad_1 docteat describe: ")
display(df_train_f1_squad_1.describe())
print("df_dev_f1_squad_1 docteat describe: ")
display(df_dev_f1_squad_1.describe())
print("df_dev_f1_squad_1 docteat describe: ")
display(df_dev_f1_squad_1.describe())

print("df_train_f2_squad_1 docteat describe: ")
display(df_train_f2_squad_1.describe())
print("df_zeroshot_f1_squad_1 docteat describe: ")
display(df_zeroshot_f1_squad_1.describe())

df_train_f1_squad_1.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_squad_1_docteat.parquet')
df_dev_f1_squad_1.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_squad_1_docteat.parquet')
df_zeroshot_f1_squad_1.to_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_squad_1_docteat.parquet')

df_train_f2_squad_1.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_squad_1_docteat.parquet')
df_dev_f2_squad_1.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_squad_1_docteat.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_squad_1_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_squad_1_docteat.parquet'),
        "zeroshot": Dataset.from_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_squad_1_docteat.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_squad_1_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_squad_1_docteat.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_SQUAD_1_DOCTEAT")

df_train_f1_squad_1 docteat describe: 


,prompt lenght
count,10058.000000
mean,498.707298
std,398.739365
min,40.000000
25%,229.000000
50%,405.000000
75%,653.000000
max,7357.000000


df_dev_f1_squad_1 docteat describe: 


,prompt lenght
count,1347.000000
mean,486.135857
std,352.351673
min,39.000000
25%,235.000000
50%,400.000000
75%,627.000000
max,3444.000000


df_dev_f1_squad_1 docteat describe: 


...

Saving the dataset (0/1 shards):   0%|          | 0/10058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1347 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/926 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10066 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1339 [00:00<?, ? examples/s]

In [35]:
df_train_f1_squad_2 = df_train_tdms_f1_docteat[df_train_tdms_f1_docteat.template == "squad_2"]
                 
df_dev_f1_squad_2 = df_dev_tdms_f1_docteat[df_dev_tdms_f1_docteat.template == "squad_2"]

df_zeroshot_f1_squad_2 = df_zeroshot_tdms_f1_docteat[df_zeroshot_tdms_f1_docteat.template == "squad_2"]

df_train_f2_squad_2 = df_train_tdms_f2_docteat[df_train_tdms_f2_docteat.template == "squad_2"]
df_dev_f2_squad_2 = df_dev_tdms_f2_docteat[df_dev_tdms_f2_docteat.template == "squad_2"]

print("df_train_f1_squad_2 docteat describe: ")
display(df_train_f1_squad_2.describe())
print("df_dev_f1_squad_2 docteat describe: ")
display(df_dev_f1_squad_2.describe())
print("df_dev_f1_squad_2 docteat describe: ")
display(df_dev_f1_squad_2.describe())

print("df_train_f2_squad_2 docteat describe: ")
display(df_train_f2_squad_2.describe())
print("df_zeroshot_f1_squad_2 docteat describe: ")
display(df_zeroshot_f1_squad_2.describe())

df_train_f1_squad_2.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_squad_2_docteat.parquet')
df_dev_f1_squad_2.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_squad_2_docteat.parquet')
df_zeroshot_f1_squad_2.to_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_squad_2_docteat.parquet')

df_train_f2_squad_2.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_squad_2_docteat.parquet')
df_dev_f2_squad_2.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_squad_2_docteat.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_squad_2_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_squad_2_docteat.parquet'),
        "zeroshot": Dataset.from_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_squad_2_docteat.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_squad_2_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_squad_2_docteat.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_SQUAD_2_DOCTEAT")

df_train_f1_squad_2 docteat describe: 


,prompt lenght
count,10058.000000
mean,497.707298
std,398.739365
min,39.000000
25%,228.000000
50%,404.000000
75%,652.000000
max,7356.000000


df_dev_f1_squad_2 docteat describe: 


,prompt lenght
count,1347.000000
mean,485.135857
std,352.351673
min,38.000000
25%,234.000000
50%,399.000000
75%,626.000000
max,3443.000000


df_dev_f1_squad_2 docteat describe: 


...

Saving the dataset (0/1 shards):   0%|          | 0/10058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1347 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/926 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10066 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1339 [00:00<?, ? examples/s]

In [36]:
df_train_f1_squad_3 = df_train_tdms_f1_docteat[df_train_tdms_f1_docteat.template == "squad_3"]
                 
df_dev_f1_squad_3 = df_dev_tdms_f1_docteat[df_dev_tdms_f1_docteat.template == "squad_3"]

df_zeroshot_f1_squad_3 = df_zeroshot_tdms_f1_docteat[df_zeroshot_tdms_f1_docteat.template == "squad_3"]

df_train_f2_squad_3 = df_train_tdms_f2_docteat[df_train_tdms_f2_docteat.template == "squad_3"]
df_dev_f2_squad_3 = df_dev_tdms_f2_docteat[df_dev_tdms_f2_docteat.template == "squad_3"]

print("df_train_f1_squad_3 docteat describe: ")
display(df_train_f1_squad_3.describe())
print("df_dev_f1_squad_3 docteat describe: ")
display(df_dev_f1_squad_3.describe())
print("df_dev_f1_squad_3 docteat describe: ")
display(df_dev_f1_squad_3.describe())

print("df_train_f2_squad_3 docteat describe: ")
display(df_train_f2_squad_3.describe())
print("df_zeroshot_f1_squad_3 docteat describe: ")
display(df_zeroshot_f1_squad_3.describe())

df_train_f1_squad_3.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_squad_3_docteat.parquet')
df_dev_f1_squad_3.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_squad_3_docteat.parquet')
df_zeroshot_f1_squad_3.to_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_squad_3_docteat.parquet')

df_train_f2_squad_3.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_squad_3_docteat.parquet')
df_dev_f2_squad_3.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_squad_3_docteat.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_squad_3_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_squad_3_docteat.parquet'),
        "zeroshot": Dataset.from_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_squad_3_docteat.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_squad_3_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_squad_3_docteat.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_SQUAD_3_DOCTEAT")

df_train_f1_squad_3 docteat describe: 


,prompt lenght
count,10058.000000
mean,491.707298
std,398.739365
min,33.000000
25%,222.000000
50%,398.000000
75%,646.000000
max,7350.000000


df_dev_f1_squad_3 docteat describe: 


,prompt lenght
count,1347.000000
mean,479.135857
std,352.351673
min,32.000000
25%,228.000000
50%,393.000000
75%,620.000000
max,3437.000000


df_dev_f1_squad_3 docteat describe: 


...

Saving the dataset (0/1 shards):   0%|          | 0/10058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1347 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/926 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10066 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1339 [00:00<?, ? examples/s]

In [37]:
df_train_f1_squad_4 = df_train_tdms_f1_docteat[df_train_tdms_f1_docteat.template == "squad_4"]
                 
df_dev_f1_squad_4 = df_dev_tdms_f1_docteat[df_dev_tdms_f1_docteat.template == "squad_4"]

df_zeroshot_f1_squad_4 = df_zeroshot_tdms_f1_docteat[df_zeroshot_tdms_f1_docteat.template == "squad_4"]

df_train_f2_squad_4 = df_train_tdms_f2_docteat[df_train_tdms_f2_docteat.template == "squad_4"]
df_dev_f2_squad_4 = df_dev_tdms_f2_docteat[df_dev_tdms_f2_docteat.template == "squad_4"]

print("df_train_f1_squad_4 docteat describe: ")
display(df_train_f1_squad_4.describe())
print("df_dev_f1_squad_4 docteat describe: ")
display(df_dev_f1_squad_4.describe())
print("df_dev_f1_squad_4 docteat describe: ")
display(df_dev_f1_squad_4.describe())

print("df_train_f2_squad_4 docteat describe: ")
display(df_train_f2_squad_4.describe())
print("df_zeroshot_f1_squad_4 docteat describe: ")
display(df_zeroshot_f1_squad_4.describe())

df_train_f1_squad_4.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_squad_4_docteat.parquet')
df_dev_f1_squad_4.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_squad_4_docteat.parquet')
df_zeroshot_f1_squad_4.to_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_squad_4_docteat.parquet')

df_train_f2_squad_4.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_squad_4_docteat.parquet')
df_dev_f2_squad_4.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_squad_4_docteat.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_squad_4_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_squad_4_docteat.parquet'),
        "zeroshot": Dataset.from_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_squad_4_docteat.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_squad_4_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_squad_4_docteat.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_SQUAD_4_DOCTEAT")

df_train_f1_squad_4 docteat describe: 


,prompt lenght
count,10058.000000
mean,494.707298
std,398.739365
min,36.000000
25%,225.000000
50%,401.000000
75%,649.000000
max,7353.000000


df_dev_f1_squad_4 docteat describe: 


,prompt lenght
count,1347.000000
mean,482.135857
std,352.351673
min,35.000000
25%,231.000000
50%,396.000000
75%,623.000000
max,3440.000000


df_dev_f1_squad_4 docteat describe: 


...

Saving the dataset (0/1 shards):   0%|          | 0/10058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1347 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/926 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10066 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1339 [00:00<?, ? examples/s]

In [38]:
df_train_f1_squad_5 = df_train_tdms_f1_docteat[df_train_tdms_f1_docteat.template == "squad_5"]
                 
df_dev_f1_squad_5 = df_dev_tdms_f1_docteat[df_dev_tdms_f1_docteat.template == "squad_5"]

df_zeroshot_f1_squad_5 = df_zeroshot_tdms_f1_docteat[df_zeroshot_tdms_f1_docteat.template == "squad_5"]

df_train_f2_squad_5 = df_train_tdms_f2_docteat[df_train_tdms_f2_docteat.template == "squad_5"]
df_dev_f2_squad_5 = df_dev_tdms_f2_docteat[df_dev_tdms_f2_docteat.template == "squad_5"]

print("df_train_f1_squad_5 docteat describe: ")
display(df_train_f1_squad_5.describe())
print("df_dev_f1_squad_5 docteat describe: ")
display(df_dev_f1_squad_5.describe())
print("df_dev_f1_squad_5 docteat describe: ")
display(df_dev_f1_squad_5.describe())

print("df_train_f2_squad_5 docteat describe: ")
display(df_train_f2_squad_5.describe())
print("df_zeroshot_f1_squad_5 docteat describe: ")
display(df_zeroshot_f1_squad_5.describe())

df_train_f1_squad_5.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_squad_5_docteat.parquet')
df_dev_f1_squad_5.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_squad_5_docteat.parquet')
df_zeroshot_f1_squad_5.to_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_squad_5_docteat.parquet')

df_train_f2_squad_5.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_squad_5_docteat.parquet')
df_dev_f2_squad_5.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_squad_5_docteat.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_squad_5_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_squad_5_docteat.parquet'),
        "zeroshot": Dataset.from_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_squad_5_docteat.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_squad_5_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_squad_5_docteat.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_SQUAD_5_DOCTEAT")

df_train_f1_squad_5 docteat describe: 


,prompt lenght
count,10058.000000
mean,499.707298
std,398.739365
min,41.000000
25%,230.000000
50%,406.000000
75%,654.000000
max,7358.000000


df_dev_f1_squad_5 docteat describe: 


,prompt lenght
count,1347.000000
mean,487.135857
std,352.351673
min,40.000000
25%,236.000000
50%,401.000000
75%,628.000000
max,3445.000000


df_dev_f1_squad_5 docteat describe: 


...

Saving the dataset (0/1 shards):   0%|          | 0/10058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1347 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/926 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10066 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1339 [00:00<?, ? examples/s]

In [39]:
df_train_f1_squad_6 = df_train_tdms_f1_docteat[df_train_tdms_f1_docteat.template == "squad_6"]
                 
df_dev_f1_squad_6 = df_dev_tdms_f1_docteat[df_dev_tdms_f1_docteat.template == "squad_6"]

df_zeroshot_f1_squad_6 = df_zeroshot_tdms_f1_docteat[df_zeroshot_tdms_f1_docteat.template == "squad_6"]

df_train_f2_squad_6 = df_train_tdms_f2_docteat[df_train_tdms_f2_docteat.template == "squad_6"]
df_dev_f2_squad_6 = df_dev_tdms_f2_docteat[df_dev_tdms_f2_docteat.template == "squad_6"]

print("df_train_f1_squad_6 docteat describe: ")
display(df_train_f1_squad_6.describe())
print("df_dev_f1_squad_6 docteat describe: ")
display(df_dev_f1_squad_6.describe())
print("df_dev_f1_squad_6 docteat describe: ")
display(df_dev_f1_squad_6.describe())

print("df_train_f2_squad_6 docteat describe: ")
display(df_train_f2_squad_6.describe())
print("df_zeroshot_f1_squad_6 docteat describe: ")
display(df_zeroshot_f1_squad_6.describe())

df_train_f1_squad_6.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_squad_6_docteat.parquet')
df_dev_f1_squad_6.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_squad_6_docteat.parquet')
df_zeroshot_f1_squad_6.to_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_squad_6_docteat.parquet')

df_train_f2_squad_6.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_squad_6_docteat.parquet')
df_dev_f2_squad_6.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_squad_6_docteat.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_squad_6_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_squad_6_docteat.parquet'),
        "zeroshot": Dataset.from_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_squad_6_docteat.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_squad_6_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_squad_6_docteat.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_SQUAD_6_DOCTEAT")

df_train_f1_squad_6 docteat describe: 


,prompt lenght
count,10058.000000
mean,493.707298
std,398.739365
min,35.000000
25%,224.000000
50%,400.000000
75%,648.000000
max,7352.000000


df_dev_f1_squad_6 docteat describe: 


,prompt lenght
count,1347.000000
mean,481.135857
std,352.351673
min,34.000000
25%,230.000000
50%,395.000000
75%,622.000000
max,3439.000000


df_dev_f1_squad_6 docteat describe: 


...

Saving the dataset (0/1 shards):   0%|          | 0/10058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1347 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/926 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10066 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1339 [00:00<?, ? examples/s]

In [40]:
df_train_f1_squad_7 = df_train_tdms_f1_docteat[df_train_tdms_f1_docteat.template == "squad_7"]
                 
df_dev_f1_squad_7 = df_dev_tdms_f1_docteat[df_dev_tdms_f1_docteat.template == "squad_7"]

df_zeroshot_f1_squad_7 = df_zeroshot_tdms_f1_docteat[df_zeroshot_tdms_f1_docteat.template == "squad_7"]

df_train_f2_squad_7 = df_train_tdms_f2_docteat[df_train_tdms_f2_docteat.template == "squad_7"]
df_dev_f2_squad_7 = df_dev_tdms_f2_docteat[df_dev_tdms_f2_docteat.template == "squad_7"]

print("df_train_f1_squad_7 docteat describe: ")
display(df_train_f1_squad_7.describe())
print("df_dev_f1_squad_7 docteat describe: ")
display(df_dev_f1_squad_7.describe())
print("df_dev_f1_squad_7 docteat describe: ")
display(df_dev_f1_squad_7.describe())

print("df_train_f2_squad_7 docteat describe: ")
display(df_train_f2_squad_7.describe())
print("df_zeroshot_f1_squad_7 docteat describe: ")
display(df_zeroshot_f1_squad_7.describe())

df_train_f1_squad_7.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_squad_7_docteat.parquet')
df_dev_f1_squad_7.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_squad_7_docteat.parquet')
df_zeroshot_f1_squad_7.to_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_squad_7_docteat.parquet')

df_train_f2_squad_7.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_squad_7_docteat.parquet')
df_dev_f2_squad_7.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_squad_7_docteat.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_squad_7_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_squad_7_docteat.parquet'),
        "zeroshot": Dataset.from_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_squad_7_docteat.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_squad_7_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_squad_7_docteat.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_SQUAD_7_DOCTEAT")

df_train_f1_squad_7 docteat describe: 


,prompt lenght
count,10058.000000
mean,497.707298
std,398.739365
min,39.000000
25%,228.000000
50%,404.000000
75%,652.000000
max,7356.000000


df_dev_f1_squad_7 docteat describe: 


,prompt lenght
count,1347.000000
mean,485.135857
std,352.351673
min,38.000000
25%,234.000000
50%,399.000000
75%,626.000000
max,3443.000000


df_dev_f1_squad_7 docteat describe: 


...

Saving the dataset (0/1 shards):   0%|          | 0/10058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1347 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/926 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10066 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1339 [00:00<?, ? examples/s]

In [41]:
df_train_f1_squad_8 = df_train_tdms_f1_docteat[df_train_tdms_f1_docteat.template == "squad_8"]
                 
df_dev_f1_squad_8 = df_dev_tdms_f1_docteat[df_dev_tdms_f1_docteat.template == "squad_8"]

df_zeroshot_f1_squad_8 = df_zeroshot_tdms_f1_docteat[df_zeroshot_tdms_f1_docteat.template == "squad_8"]

df_train_f2_squad_8 = df_train_tdms_f2_docteat[df_train_tdms_f2_docteat.template == "squad_8"]
df_dev_f2_squad_8 = df_dev_tdms_f2_docteat[df_dev_tdms_f2_docteat.template == "squad_8"]

print("df_train_f1_squad_8 docteat describe: ")
display(df_train_f1_squad_8.describe())
print("df_dev_f1_squad_8 docteat describe: ")
display(df_dev_f1_squad_8.describe())
print("df_dev_f1_squad_8 docteat describe: ")
display(df_dev_f1_squad_8.describe())

print("df_train_f2_squad_8 docteat describe: ")
display(df_train_f2_squad_8.describe())
print("df_zeroshot_f1_squad_8 docteat describe: ")
display(df_zeroshot_f1_squad_8.describe())

df_train_f1_squad_8.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_squad_8_docteat.parquet')
df_dev_f1_squad_8.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_squad_8_docteat.parquet')
df_zeroshot_f1_squad_8.to_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_squad_8_docteat.parquet')

df_train_f2_squad_8.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_squad_8_docteat.parquet')
df_dev_f2_squad_8.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_squad_8_docteat.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_squad_8_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_squad_8_docteat.parquet'),
        "zeroshot": Dataset.from_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_squad_8_docteat.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_squad_8_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_squad_8_docteat.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_SQUAD_8_DOCTEAT")

df_train_f1_squad_8 docteat describe: 


,prompt lenght
count,10058.000000
mean,502.707298
std,398.739365
min,44.000000
25%,233.000000
50%,409.000000
75%,657.000000
max,7361.000000


df_dev_f1_squad_8 docteat describe: 


,prompt lenght
count,1347.000000
mean,490.135857
std,352.351673
min,43.000000
25%,239.000000
50%,404.000000
75%,631.000000
max,3448.000000


df_dev_f1_squad_8 docteat describe: 


...

Saving the dataset (0/1 shards):   0%|          | 0/10058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1347 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/926 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10066 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1339 [00:00<?, ? examples/s]

## DROP

In [42]:
df_train_f1_drop_1 = df_train_tdms_f1_docteat[df_train_tdms_f1_docteat.template == "drop_1"]
                 
df_dev_f1_drop_1 = df_dev_tdms_f1_docteat[df_dev_tdms_f1_docteat.template == "drop_1"]

df_zeroshot_f1_drop_1 = df_zeroshot_tdms_f1_docteat[df_zeroshot_tdms_f1_docteat.template == "drop_1"]

df_train_f2_drop_1 = df_train_tdms_f2_docteat[df_train_tdms_f2_docteat.template == "drop_1"]
df_dev_f2_drop_1 = df_dev_tdms_f2_docteat[df_dev_tdms_f2_docteat.template == "drop_1"]

print("df_train_f1_drop_1 docteat describe: ")
display(df_train_f1_drop_1.describe())
print("df_dev_f1_drop_1 docteat describe: ")
display(df_dev_f1_drop_1.describe())
print("df_dev_f1_drop_1 docteat describe: ")
display(df_dev_f1_drop_1.describe())

print("df_train_f2_drop_1 docteat describe: ")
display(df_train_f2_drop_1.describe())
print("df_zeroshot_f1_drop_1 docteat describe: ")
display(df_zeroshot_f1_drop_1.describe())

df_train_f1_drop_1.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_drop_1_docteat.parquet')
df_dev_f1_drop_1.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_drop_1_docteat.parquet')
df_zeroshot_f1_drop_1.to_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_drop_1_docteat.parquet')

df_train_f2_drop_1.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_drop_1_docteat.parquet')
df_dev_f2_drop_1.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_drop_1_docteat.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_drop_1_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_drop_1_docteat.parquet'),
        "zeroshot": Dataset.from_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_drop_1_docteat.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_drop_1_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_drop_1_docteat.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_DROP_1_DOCTEAT")

df_train_f1_drop_1 docteat describe: 


,prompt lenght
count,10058.000000
mean,488.707298
std,398.739365
min,30.000000
25%,219.000000
50%,395.000000
75%,643.000000
max,7347.000000


df_dev_f1_drop_1 docteat describe: 


,prompt lenght
count,1347.000000
mean,476.135857
std,352.351673
min,29.000000
25%,225.000000
50%,390.000000
75%,617.000000
max,3434.000000


df_dev_f1_drop_1 docteat describe: 


...

Saving the dataset (0/1 shards):   0%|          | 0/10058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1347 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/926 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10066 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1339 [00:00<?, ? examples/s]

In [43]:
df_train_f1_drop_2 = df_train_tdms_f1_docteat[df_train_tdms_f1_docteat.template == "drop_2"]
                 
df_dev_f1_drop_2 = df_dev_tdms_f1_docteat[df_dev_tdms_f1_docteat.template == "drop_2"]

df_zeroshot_f1_drop_2 = df_zeroshot_tdms_f1_docteat[df_zeroshot_tdms_f1_docteat.template == "drop_2"]

df_train_f2_drop_2 = df_train_tdms_f2_docteat[df_train_tdms_f2_docteat.template == "drop_2"]
df_dev_f2_drop_2 = df_dev_tdms_f2_docteat[df_dev_tdms_f2_docteat.template == "drop_2"]

print("df_train_f1_drop_2 docteat describe: ")
display(df_train_f1_drop_2.describe())
print("df_dev_f1_drop_2 docteat describe: ")
display(df_dev_f1_drop_2.describe())
print("df_dev_f1_drop_2 docteat describe: ")
display(df_dev_f1_drop_2.describe())

print("df_train_f2_drop_2 docteat describe: ")
display(df_train_f2_drop_2.describe())
print("df_zeroshot_f1_drop_2 docteat describe: ")
display(df_zeroshot_f1_drop_2.describe())

df_train_f1_drop_2.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_drop_2_docteat.parquet')
df_dev_f1_drop_2.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_drop_2_docteat.parquet')
df_zeroshot_f1_drop_2.to_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_drop_2_docteat.parquet')

df_train_f2_drop_2.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_drop_2_docteat.parquet')
df_dev_f2_drop_2.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_drop_2_docteat.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_drop_2_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_drop_2_docteat.parquet'),
        "zeroshot": Dataset.from_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_drop_2_docteat.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_drop_2_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_drop_2_docteat.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_DROP_2_DOCTEAT")

df_train_f1_drop_2 docteat describe: 


,prompt lenght
count,10058.000000
mean,491.707298
std,398.739365
min,33.000000
25%,222.000000
50%,398.000000
75%,646.000000
max,7350.000000


df_dev_f1_drop_2 docteat describe: 


,prompt lenght
count,1347.000000
mean,479.135857
std,352.351673
min,32.000000
25%,228.000000
50%,393.000000
75%,620.000000
max,3437.000000


df_dev_f1_drop_2 docteat describe: 


...

Saving the dataset (0/1 shards):   0%|          | 0/10058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1347 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/926 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10066 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1339 [00:00<?, ? examples/s]

In [44]:
df_train_f1_drop_3 = df_train_tdms_f1_docteat[df_train_tdms_f1_docteat.template == "drop_3"]
                 
df_dev_f1_drop_3 = df_dev_tdms_f1_docteat[df_dev_tdms_f1_docteat.template == "drop_3"]

df_zeroshot_f1_drop_3 = df_zeroshot_tdms_f1_docteat[df_zeroshot_tdms_f1_docteat.template == "drop_3"]

df_train_f2_drop_3 = df_train_tdms_f2_docteat[df_train_tdms_f2_docteat.template == "drop_3"]
df_dev_f2_drop_3 = df_dev_tdms_f2_docteat[df_dev_tdms_f2_docteat.template == "drop_3"]

print("df_train_f1_drop_3 docteat describe: ")
display(df_train_f1_drop_3.describe())
print("df_dev_f1_drop_3 docteat describe: ")
display(df_dev_f1_drop_3.describe())
print("df_dev_f1_drop_3 docteat describe: ")
display(df_dev_f1_drop_3.describe())

print("df_train_f2_drop_3 docteat describe: ")
display(df_train_f2_drop_3.describe())
print("df_zeroshot_f1_drop_3 docteat describe: ")
display(df_zeroshot_f1_drop_3.describe())

df_train_f1_drop_3.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_drop_3_docteat.parquet')
df_dev_f1_drop_3.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_drop_3_docteat.parquet')
df_zeroshot_f1_drop_3.to_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_drop_3_docteat.parquet')

df_train_f2_drop_3.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_drop_3_docteat.parquet')
df_dev_f2_drop_3.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_drop_3_docteat.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_drop_3_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_drop_3_docteat.parquet'),
        "zeroshot": Dataset.from_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_drop_3_docteat.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_drop_3_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_drop_3_docteat.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_DROP_3_DOCTEAT")

df_train_f1_drop_3 docteat describe: 


,prompt lenght
count,10051.000000
mean,484.701622
std,398.858367
min,26.000000
25%,215.000000
50%,391.000000
75%,639.500000
max,7343.000000


df_dev_f1_drop_3 docteat describe: 


,prompt lenght
count,1344.000000
mean,472.113839
std,352.657013
min,25.000000
25%,221.000000
50%,385.000000
75%,613.000000
max,3430.000000


df_dev_f1_drop_3 docteat describe: 


...

Saving the dataset (0/1 shards):   0%|          | 0/10051 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1344 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/924 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1337 [00:00<?, ? examples/s]

In [45]:
df_train_f1_drop_4 = df_train_tdms_f1_docteat[df_train_tdms_f1_docteat.template == "drop_4"]
                 
df_dev_f1_drop_4 = df_dev_tdms_f1_docteat[df_dev_tdms_f1_docteat.template == "drop_4"]

df_zeroshot_f1_drop_4 = df_zeroshot_tdms_f1_docteat[df_zeroshot_tdms_f1_docteat.template == "drop_4"]

df_train_f2_drop_4 = df_train_tdms_f2_docteat[df_train_tdms_f2_docteat.template == "drop_4"]
df_dev_f2_drop_4 = df_dev_tdms_f2_docteat[df_dev_tdms_f2_docteat.template == "drop_4"]

print("df_train_f1_drop_4 docteat describe: ")
display(df_train_f1_drop_4.describe())
print("df_dev_f1_drop_4 docteat describe: ")
display(df_dev_f1_drop_4.describe())
print("df_dev_f1_drop_4 docteat describe: ")
display(df_dev_f1_drop_4.describe())

print("df_train_f2_drop_4 docteat describe: ")
display(df_train_f2_drop_4.describe())
print("df_zeroshot_f1_drop_4 docteat describe: ")
display(df_zeroshot_f1_drop_4.describe())

df_train_f1_drop_4.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_drop_4_docteat.parquet')
df_dev_f1_drop_4.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_drop_4_docteat.parquet')
df_zeroshot_f1_drop_4.to_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_drop_4_docteat.parquet')

df_train_f2_drop_4.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_drop_4_docteat.parquet')
df_dev_f2_drop_4.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_drop_4_docteat.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_drop_4_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_drop_4_docteat.parquet'),
        "zeroshot": Dataset.from_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_drop_4_docteat.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_drop_4_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_drop_4_docteat.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_DROP_4_DOCTEAT")

df_train_f1_drop_4 docteat describe: 


,prompt lenght
count,10058.000000
mean,487.707298
std,398.739365
min,29.000000
25%,218.000000
50%,394.000000
75%,642.000000
max,7346.000000


df_dev_f1_drop_4 docteat describe: 


,prompt lenght
count,1347.000000
mean,475.135857
std,352.351673
min,28.000000
25%,224.000000
50%,389.000000
75%,616.000000
max,3433.000000


df_dev_f1_drop_4 docteat describe: 


...

Saving the dataset (0/1 shards):   0%|          | 0/10058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1347 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/926 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10066 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1339 [00:00<?, ? examples/s]

In [46]:
df_train_f1_drop_5 = df_train_tdms_f1_docteat[df_train_tdms_f1_docteat.template == "drop_5"]
                 
df_dev_f1_drop_5 = df_dev_tdms_f1_docteat[df_dev_tdms_f1_docteat.template == "drop_5"]

df_zeroshot_f1_drop_5 = df_zeroshot_tdms_f1_docteat[df_zeroshot_tdms_f1_docteat.template == "drop_5"]

df_train_f2_drop_5 = df_train_tdms_f2_docteat[df_train_tdms_f2_docteat.template == "drop_5"]
df_dev_f2_drop_5 = df_dev_tdms_f2_docteat[df_dev_tdms_f2_docteat.template == "drop_5"]

print("df_train_f1_drop_5 docteat describe: ")
display(df_train_f1_drop_5.describe())
print("df_dev_f1_drop_5 docteat describe: ")
display(df_dev_f1_drop_5.describe())
print("df_dev_f1_drop_5 docteat describe: ")
display(df_dev_f1_drop_5.describe())

print("df_train_f2_drop_5 docteat describe: ")
display(df_train_f2_drop_5.describe())
print("df_zeroshot_f1_drop_5 docteat describe: ")
display(df_zeroshot_f1_drop_5.describe())

df_train_f1_drop_5.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_drop_5_docteat.parquet')
df_dev_f1_drop_5.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_drop_5_docteat.parquet')
df_zeroshot_f1_drop_5.to_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_drop_5_docteat.parquet')

df_train_f2_drop_5.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_drop_5_docteat.parquet')
df_dev_f2_drop_5.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_drop_5_docteat.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_drop_5_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_drop_5_docteat.parquet'),
        "zeroshot": Dataset.from_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_drop_5_docteat.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_drop_5_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_drop_5_docteat.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_DROP_5_DOCTEAT")

df_train_f1_drop_5 docteat describe: 


,prompt lenght
count,10058.000000
mean,491.707298
std,398.739365
min,33.000000
25%,222.000000
50%,398.000000
75%,646.000000
max,7350.000000


df_dev_f1_drop_5 docteat describe: 


,prompt lenght
count,1347.000000
mean,479.135857
std,352.351673
min,32.000000
25%,228.000000
50%,393.000000
75%,620.000000
max,3437.000000


df_dev_f1_drop_5 docteat describe: 


...

Saving the dataset (0/1 shards):   0%|          | 0/10058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1347 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/926 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10066 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1339 [00:00<?, ? examples/s]

In [47]:
df_train_f1_drop_6 = df_train_tdms_f1_docteat[df_train_tdms_f1_docteat.template == "drop_6"]
                 
df_dev_f1_drop_6 = df_dev_tdms_f1_docteat[df_dev_tdms_f1_docteat.template == "drop_6"]

df_zeroshot_f1_drop_6 = df_zeroshot_tdms_f1_docteat[df_zeroshot_tdms_f1_docteat.template == "drop_6"]

df_train_f2_drop_6 = df_train_tdms_f2_docteat[df_train_tdms_f2_docteat.template == "drop_6"]
df_dev_f2_drop_6 = df_dev_tdms_f2_docteat[df_dev_tdms_f2_docteat.template == "drop_6"]

print("df_train_f1_drop_6 docteat describe: ")
display(df_train_f1_drop_6.describe())
print("df_dev_f1_drop_6 docteat describe: ")
display(df_dev_f1_drop_6.describe())
print("df_dev_f1_drop_6 docteat describe: ")
display(df_dev_f1_drop_6.describe())

print("df_train_f2_drop_6 docteat describe: ")
display(df_train_f2_drop_6.describe())
print("df_zeroshot_f1_drop_6 docteat describe: ")
display(df_zeroshot_f1_drop_6.describe())

df_train_f1_drop_6.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_drop_6_docteat.parquet')
df_dev_f1_drop_6.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_drop_6_docteat.parquet')
df_zeroshot_f1_drop_6.to_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_drop_6_docteat.parquet')

df_train_f2_drop_6.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_drop_6_docteat.parquet')
df_dev_f2_drop_6.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_drop_6_docteat.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_drop_6_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_drop_6_docteat.parquet'),
        "zeroshot": Dataset.from_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_drop_6_docteat.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_drop_6_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_drop_6_docteat.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_DROP_6_DOCTEAT")

df_train_f1_drop_6 docteat describe: 


,prompt lenght
count,10058.000000
mean,492.707298
std,398.739365
min,34.000000
25%,223.000000
50%,399.000000
75%,647.000000
max,7351.000000


df_dev_f1_drop_6 docteat describe: 


,prompt lenght
count,1347.000000
mean,480.135857
std,352.351673
min,33.000000
25%,229.000000
50%,394.000000
75%,621.000000
max,3438.000000


df_dev_f1_drop_6 docteat describe: 


...

Saving the dataset (0/1 shards):   0%|          | 0/10058 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1347 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/926 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10066 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1339 [00:00<?, ? examples/s]

In [48]:
df_train_f1_drop_7 = df_train_tdms_f1_docteat[df_train_tdms_f1_docteat.template == "drop_7"]
                 
df_dev_f1_drop_7 = df_dev_tdms_f1_docteat[df_dev_tdms_f1_docteat.template == "drop_7"]

df_zeroshot_f1_drop_7 = df_zeroshot_tdms_f1_docteat[df_zeroshot_tdms_f1_docteat.template == "drop_7"]

df_train_f2_drop_7 = df_train_tdms_f2_docteat[df_train_tdms_f2_docteat.template == "drop_7"]
df_dev_f2_drop_7 = df_dev_tdms_f2_docteat[df_dev_tdms_f2_docteat.template == "drop_7"]

print("df_train_f1_drop_7 docteat describe: ")
display(df_train_f1_drop_7.describe())
print("df_dev_f1_drop_7 docteat describe: ")
display(df_dev_f1_drop_7.describe())
print("df_dev_f1_drop_7 docteat describe: ")
display(df_dev_f1_drop_7.describe())

print("df_train_f2_drop_7 docteat describe: ")
display(df_train_f2_drop_7.describe())
print("df_zeroshot_f1_drop_7 docteat describe: ")
display(df_zeroshot_f1_drop_7.describe())

df_train_f1_drop_7.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_drop_7_docteat.parquet')
df_dev_f1_drop_7.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_drop_7_docteat.parquet')
df_zeroshot_f1_drop_7.to_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_drop_7_docteat.parquet')

df_train_f2_drop_7.to_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_drop_7_docteat.parquet')
df_dev_f2_drop_7.to_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_drop_7_docteat.parquet')

dataset = DatasetDict({
    'fold1': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f1_drop_7_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f1_drop_7_docteat.parquet'),
        "zeroshot": Dataset.from_parquet('../data/df_zeroshot_tdms_augmented_summarized_with_id_f1_drop_7_docteat.parquet')
    }),
    'fold2': DatasetDict({
        "train": Dataset.from_parquet('../data/df_train_tdms_augmented_summarized_with_id_f2_drop_7_docteat.parquet'),
        "validation": Dataset.from_parquet('../data/df_dev_tdms_augmented_summarized_with_id_f2_drop_7_docteat.parquet')
    })
})

print(dataset)

dataset.save_to_disk("../data/LLLM_AUGMENTED_SUMMARIZED_WITH_ID_ZEROSHOT_TDMS_DROP_7_DOCTEAT")

df_train_f1_drop_7 docteat describe: 


,prompt lenght
count,10065.000000
mean,487.712966
std,398.620494
min,29.000000
25%,218.000000
50%,394.000000
75%,642.000000
max,7346.000000


df_dev_f1_drop_7 docteat describe: 


,prompt lenght
count,1350.000000
mean,475.157778
std,352.047426
min,28.000000
25%,224.250000
50%,389.000000
75%,615.750000
max,3433.000000


df_dev_f1_drop_7 docteat describe: 


...

Saving the dataset (0/1 shards):   0%|          | 0/10065 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1350 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/928 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10074 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1341 [00:00<?, ? examples/s]

In [49]:
2+2

4

## No templates data

In [5]:
# root_directory = "../data/LLLM_LONG_SUMMARIZED_TDMS_ALL_TEMPLATE"
root_directory = "../data/LLLM_AUGMENTED_SUMMARIZED_ZEROSHOT_TDMS_50_PERCENT_DOCTEAT"


# reloaded_encoded_dataset = datasets.load_from_disk("../data/dataset/LLLM_TDMS_ALL_TEMPLATE")
# reloaded_encoded_dataset = DatasetDict.load_from_disk("../data/LLLM_TDMS_ALL_TEMPLATE")

dataset_fold1 = DatasetDict.load_from_disk(f"{root_directory}/fold1")
dataset_fold2 = DatasetDict.load_from_disk(f"{root_directory}/fold2")

dataset_dict = DatasetDict.load_from_disk(f"{root_directory}/fold1")

In [3]:
dataset_fold1['train'][0]

{'id': '2005.10469v1.pdf',
 'prompt': 'ASAPP-ASR: Multistream CNN and Self-Attentive SRUfor SOTA Speech Recognition In this paper we present state-of-the-art (SOTA) performance on the LibriSpeech corpus with two novel neural network architectures, a multistream CNN for acoustic modeling and a self-attentive simple recurrent unit (SRU) for language modeling. In the hybrid ASR framework, the multistream CNN acoustic model processes an input of speech frames in multiple parallel pipelines where each stream has a unique dilation rate for diversity. Trained with the SpecAugment data augmentation method, it achieves relative word error rate (WER) improvements of 4% on test-clean and 14% on test-other. We further improve the performance via -best rescoring using a 24-layer self-attentive SRU language model, achieving WERs of 1.75% on test-clean and 4.46% on test-other. We conduct the experiments on the LibriSpeech corpus , which is a collection of approximately 1,000hr read speech (16kHz) fro

In [6]:
dfs = {split: dataset.to_pandas() for split, dataset in dataset_dict.items()}

dfs['validation'].head()

,id,prompt,answer,template,prompt lenght,__index_level_0__
0,1712.02047v1.pdf,Distance-based Self-Attention Network for Natu...,[{'LEADERBOARD': {'Task': 'Natural Language In...,squad_1,504,0
1,1508.01819v1.pdf,Spectral Clustering and Block Models: A Review...,unanswerable,squad_1,49,1
2,1909.04810v3.pdf,Antipodal Robotic Grasping usingGenerative Res...,unanswerable,squad_1,619,2
3,1601.02071.pdf,Sentiment Visualisation Widgets for Explorator...,unanswerable,squad_1,181,3
4,2008.06695v1.pdf,Label-Wise Document Pre-Training for Multi-Lab...,[{'LEADERBOARD': {'Task': 'Multi-Label Text Cl...,squad_1,860,4


In [10]:
dfs['validation'][dfs['validation']['template'] == "squad_1"].describe()

,prompt lenght,__index_level_0__
count,669.000000,669.000000
mean,484.209268,334.000000
std,356.405451,193.267949
min,39.000000,0.000000
25%,238.000000,167.000000
50%,400.000000,334.000000
75%,620.000000,501.000000
max,3444.000000,668.000000


In [9]:
dfs['validation'].describe()

,prompt lenght,__index_level_0__
count,10103.000000,10103.000000
mean,482.624270,5051.000000
std,355.591363,2916.629219
min,25.000000,0.000000
25%,231.000000,2525.500000
50%,395.000000,5051.000000
75%,622.000000,7576.500000
max,3445.000000,10102.000000


In [8]:
dfs['zeroshot'].describe()

,prompt lenght,__index_level_0__
count,6903.000000,6903.000000
mean,454.665363,3451.000000
std,417.789826,1992.868786
min,28.000000,0.000000
25%,207.000000,1725.500000
50%,352.000000,3451.000000
75%,592.000000,5176.500000
max,7361.000000,6902.000000
